In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import requests

In [2]:
url = "https://servicodados.ibge.gov.br/api/v3/agregados/1839/periodos/2022/variaveis/630?localidades=N1[all]"

In [3]:
# Extração
response = requests.get(url)
response.raise_for_status()
data = response.json()


# Extração
response = requests.get(url)
response.raise_for_status()  # Levanta um erro se a requisição falhar
data = response.json()

# Cria uma sessão Spark
spark = SparkSession.builder.appName("PesquisaIndustrialAnual").getOrCreate()

# Define o schema
schema = StructType([
    StructField("variavel_id", IntegerType(), False),
    StructField("variavel", StringType(), False),
    StructField("categoria_id", IntegerType(), False),
    StructField("categoria_nome", StringType(), False),
    StructField("unidade", StringType(), False),
    StructField("valor", IntegerType(), False),
    StructField("classificacao_id", IntegerType(), False),
    StructField("classificacao_nome", StringType(), False),
    StructField("localidade_id", IntegerType(), False),
    StructField("localidade", StringType(), False),
    StructField("nivel_geografico", StringType(), False),
    StructField("data", StringType(), False),
])

# Função para extrair os dados
def extrair_dados(data):
    linhas = []
    
    for item in data:
        variavel_id = int(item["id"])
        variavel = item["variavel"]
        unidade = item["unidade"]
        
        resultado = item["resultados"][0]
        valor = int(resultado["series"][0]["serie"]["2022"])
        
        localidade_id = int(resultado["series"][0]["localidade"]["id"])
        localidade = resultado["series"][0]["localidade"]["nome"]
        nivel_geografico = resultado["series"][0]["localidade"]["nivel"]["id"]
        data_ano = "2022"
        
        classificacoes = resultado["classificacoes"]
        
        for classificacao in classificacoes:
            classificacao_id = int(classificacao["id"])
            classificacao_nome = classificacao["nome"]
            
            for categoria_id, categoria_nome in classificacao["categoria"].items():
                linhas.append((
                    variavel_id,
                    variavel,
                    int(categoria_id),
                    categoria_nome,
                    unidade,
                    valor,
                    classificacao_id,
                    classificacao_nome,
                    localidade_id,
                    localidade,
                    nivel_geografico,
                    data_ano
                ))
    
    # Cria DataFrame do Spark
    df = spark.createDataFrame(linhas, schema=schema)
    return df

# Instancia o DataFrame
df_spark = extrair_dados(data)

# Mostrando os resultados
df_spark.show()

[{'id': '630',
  'variavel': 'Número de empresas',
  'unidade': 'Unidades',
  'resultados': [{'classificacoes': [{'id': '319',
      'nome': 'Faixas de pessoal ocupado',
      'categoria': {'104029': 'Total'}},
     {'id': '9117',
      'nome': 'Tipo de indústria',
      'categoria': {'99735': 'Total'}}],
    'series': [{'localidade': {'id': '1',
       'nivel': {'id': 'N1', 'nome': 'Brasil'},
       'nome': 'Brasil'},
      'serie': {'2022': '346105'}}]}]}]